## 채용공고 scrapping

In [2]:
import sqlite3
conn = sqlite3.connect('./db.sqlite3')
cur = conn.cursor()

In [3]:
from selenium import webdriver

In [4]:
browser = webdriver.Chrome('./chromedriver.exe')

In [5]:
browser.get('https://www.jobindexworld.com/jobpost/list')

In [6]:
html = browser.page_source

In [7]:
from bs4 import BeautifulSoup

In [8]:
soup = BeautifulSoup(html, 'html.parser')

### 채용공고 list를 클릭해서 들어가기

추가해야될 것

상세대상 URL : detail_target_url
대분류명 : category_big	# 공통 IT
중분류명 : category_middle
소분류명 : category_small
주요 검색어 : search_word  # 사용했을 시

In [18]:
import datetime
import time

for i in range(5):

    target_name = 'jobindex'
    target_url = 'https://www.jobindexworld.com/index'
    detail_target_url = None
    category_big = 'IT'
    category_middle = None
    category_small = None
    search_word = None
    
    try:
        now = datetime.datetime.now()
        create_date = now.strftime("%Y%m%d") # 스크래핑 날짜

        # click해서 들어간 창
        browser.find_elements_by_css_selector('article.cpn-circle-conts-item')[i].click() # 클릭 페이지
        time.sleep(1)
        html_scraping = browser.page_source 
        soup_scraping = BeautifulSoup(html_scraping, 'html.parser')

        tags_signs = soup_scraping.select('div.circle-conts-view-main-conts-box-question > div') # '◈' 기호와 if문을 통해 form을 선별

        recruit = soup_scraping.select('article.circle-conts-view > h3.tit-h3') # 주제, 제목
        recruit_title = recruit[0].text.strip()

        uri = soup_scraping.select('meta[property="og:url"]') # 구직 회사 내용 화면
        detail_uri = uri[0]['content']

        start_date = soup_scraping.select('div.date')
        start_date_t = start_date[0].text.strip()[1:9] # 날짜, 시간 -> data time: https://docs.python.org/ko/3/library/datetime.html
        apply_start_date = '20'+start_date_t.replace('.','')
        
        
    #---------------------------------------------------------------------------------------------------            
        # 윗태그 2개 아래태그 5개 스크래핑 양식(career_requirements 컬럼이 없으므로 ' ' 추가)
        if len(tags_signs) > 9:
            tags_li = soup_scraping.select('div.circle-conts-view-main-conts-box-question > div > ul > li')
            tags_li_data = []

            for a in range(len(tags_li)):
                tags_li_data.append(tags_li[a].text.strip().replace(' ','').replace('\n','').split(':'))

            for b in range(len(tags_li_data)):
                if tags_li_data[b][0] == '회사개요':
                    company_name = tags_li_data[b][1]
                elif tags_li_data[b][0] == '모집기간':
                    apply_end_date = tags_li_data[b][1]
            

            tags = soup_scraping.select('div.circle-conts-view-main-conts-box-question > div')
            tags = tags[1:len(tags)]
            tags_data = []

            for c in range(len(tags)):
                tags_data.append(tags[c].text.strip().replace('◈ ',''))

            for d in range(len(tags_data)):
                if tags_data[d] == '직무 개요':
                    task = tags_data[d+1]

                elif tags_data[d] == '경력 요건':
                    need_career = tags_data[d+1]

                elif tags_data[d] == '처우':
                    team_env = tags_data[d+1]

                elif tags_data[d] == '제출 서류 및 기타':
                    need_doc = tags_data[d+1]

            career_requirements = None
            print(i,'페이지' ,'1번 스크래핑 양식')

    #-------------------------------------------------------------------------------------------------
        # 윗태그 5개 스크래핑 양식(task, need_doc이 없으므로 ' '을 추가)
        else:
            tags_li = soup_scraping.select('div.circle-conts-view-main-conts-box-question > div > ul > li')
            tags_li_data = []

            for e in range(len(tags_li)):
                tags_li_data.append(tags_li[e].text.strip().replace(' ','').replace('\n','').split(':'))

            for f in range(len(tags_li_data)):
                if tags_li_data[f][0] == '회사개요':
                    company_name = tags_li_data[f][1]

                elif tags_li_data[f][0] == '경력레벨':
                    need_career = tags_li_data[f][1]

                elif tags_li_data[f][0] == '급여수준':
                    team_env = tags_li_data[f][1]

                elif tags_li_data[f][0] == '교육수준':
                    career_requirements = tags_li_data[f][1]

                elif tags_li_data[f][0] == '모집기간':
                    apply_end_date = tags_li_data[f][1]


            task = None
            need_doc = None
            print(i,'페이지','2번 스크래핑 양식')

    #---------------------------------------------------------------------------------------------------       

        contact = soup_scraping.select('span.writer-info') # 담당컨설턴트
        contact_people = contact[0].text.strip()[0:3] # 이름 뽑아내기

        hash_list = []
        hash = soup_scraping.select('div.keyword-wrap > a.keyword') # 해시태그
        for j in range(len(hash)):  # 반복문으로 해시태그들을 뽑아 리스트로 만들기
            hash_t = hash[j].text.strip()
            hash_list.append(hash_t)
        
        hashtag = ''
        for k in hash_list:
            hashtag = hashtag + k
            
        print(id, target_name, target_url, detail_target_url, category_big, category_middle, category_small, search_word, create_date, recruit_title, company_name, detail_uri, apply_start_date, apply_end_date, task, need_career, team_env, need_doc, contact_people, hashtag)    
        time.sleep(1)
        cur.execute('''insert into SCRAPPING_SITE_B(
            apply_end_date,
            apply_start_date,
            career_requirements,
            category_big,
            company_name,
            contact_people,
            detail_uri,
            hashtag,
            need_career,
            need_doc,
            recruit_title,
            target_name,
            target_url,
            task,
            team_env)
        values(?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)''', (apply_end_date, apply_start_date, career_requirements, category_big, company_name, contact_people, detail_uri, hashtag, need_career, need_doc, recruit_title, target_name, target_url, task, team_env))
        conn.commit()
        browser.back()
        # 더보기 button 클릭하기 태그 : i.icon-plus-sign-to-add
        
    except:
        pass
# conn.close()

0 페이지 2번 스크래핑 양식
<built-in function id> jobindex https://www.jobindexworld.com/index None IT None None None 20210910 운영품질 QAM(대리-과장) 국내IT사 https://www.jobindexworld.com/jobpost/view/53405 20210908 채용시까지 None mid-level,Senior,Lead 협의 None 최수현 #IT#ITSM#ITinfra#품질#QA#브레인202_최수현헤드헌터
1 페이지 2번 스크래핑 양식
<built-in function id> jobindex https://www.jobindexworld.com/index None IT None None None 20210910 SW기술품질(과장-차장) 국내IT사 https://www.jobindexworld.com/jobpost/view/53404 20210908 채용시까지 None Senior,Lead,Manager 협의 None 최수현 #IT#SW#품질#JIRA#브레인202_최수현헤드헌터
2 페이지 2번 스크래핑 양식
<built-in function id> jobindex https://www.jobindexworld.com/index None IT None None None 20210910 Compliance Manager 국내금융사 https://www.jobindexworld.com/jobpost/view/53403 20210908 채용시까지 None Senior,Lead,Manager 협의 None 최수현 #Compliance#법무#금융#은행#브레인202_최수현헤드헌터
3 페이지 2번 스크래핑 양식
<built-in function id> jobindex https://www.jobindexworld.com/index None IT None None None 20210910 DBA(대리-과장급) 국내유통사 https://www.jobindexworld.com/jobpost/

In [ ]:
# cur.execute('''CREATE TABLE SCRAPPING_SITE_B(id INTEGER,
# target_name TEXT,
# target_url TEXT,
# detail_target_url TEXT,
# category_big TEXT,
# category_middle TEXT, 
# category_small TEXT, 
# search_word TEXT, 
# recruit_title TEXT, 
# detail_uri TEXT, 
# apply_start_date TEXT, 
# apply_end_date TEXT, 
# company_name TEXT, 
# task TEXT, 
# need_career TEXT, 
# team_env TEXT, 
# need_doc TEXT, 
# career_requirements TEXT, 
# contact_people TEXT, 
# hashtag TEXT,
# PRIMARY KEY (id))''')

In [ ]:
# 참고용
#for link in links:
#     title = str.strip(link.get_text())
#     href = str.strip(link.get('href'))
#     try:
#         cursor.execute(
#             "insert into polls_economics(create_date, href, title) values(datetime('now'), ?, ?)", (href,title))
#         print(title, ' : ', href)
#     except:
#         pass

In [14]:
cur.execute('''insert into SCRAPPING_SITE_B (
    apply_end_date,
    apply_start_date,
    career_requirements,
    category_big,
    company_name,
    contact_people,
    detail_uri,
    hashtag,
    need_career,
    need_doc,
    recruit_title,
    target_name,
    target_url,
    task,
    team_env)
values (
    'apply_end_date',
    'pply_start_date',
    'career_requirements',
    'category_big',
    'company_name',
    'contact_people',
    'detail_uri',
    'hashtag',
    'need_career',
    'need_doc',
    'recruit_title',
    'target_name',
    'target_url',
    'task',
    'team_env')''')


In [16]:
cur.execute('''insert into SCRAPPING_SITE_B(
    apply_end_date,
    apply_start_date,
    career_requirements,
    category_big,
    company_name,
    contact_people,
    detail_uri,
    hashtag,
    need_career,
    need_doc,
    recruit_title,
    target_name,
    target_url,
    task,
    team_env)
values(?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)''', (apply_end_date, apply_start_date, career_requirements, category_big, company_name, contact_people, detail_uri, hashtag, need_career, need_doc, recruit_title, target_name, target_url, task, team_env))

In [19]:
cur.execute("SELECT * FROM SCRAPPING_SITE_B") # newspaper text(기사)의 내용을 추출

articles = [] # list 선언

for row in cur.fetchall():
  articles.append(row) 
articles

[(1,
  'target_name',
  'target_url',
  None,
  'category_big',
  None,
  None,
  None,
  'recruit_title',
  'detail_uri',
  'pply_start_date',
  'apply_end_date',
  'company_name',
  'task',
  'need_career',
  'team_env',
  'need_doc',
  'career_requirements',
  'contact_people',
  'hashtag'),
 (2,
  'target_name',
  'target_url',
  None,
  'category_big',
  None,
  None,
  None,
  'recruit_title',
  'detail_uri',
  'pply_start_date',
  'apply_end_date',
  'company_name',
  'task',
  'need_career',
  'team_env',
  'need_doc',
  'career_requirements',
  'contact_people',
  'hashtag'),
 (3,
  'jobindex',
  'https://www.jobindexworld.com/index',
  None,
  'IT',
  None,
  None,
  None,
  'Vision AI Engineer',
  'https://www.jobindexworld.com/jobpost/view/53304',
  '20210825',
  '채용시까지',
  '국내IT사',
  None,
  'mid-level,Senior',
  '협의',
  None,
  '학사,석사/MBA,박사',
  '최수현',
  '#AI#IT#IT_Engineer#ComputerVision#브레인202_최수현헤드헌터'),
 (4,
  'jobindex',
  'https://www.jobindexworld.com/index',
  None

In [ ]:
cur.execute('''select
    ssb.apply_end_date,
    ssb.apply_start_date,
    ssb.career_requirements,
    ssb.category_big,
    ssb.category_middle,
    ssb.category_small,
    ssb.company_name,
    ssb.contact_people,
    ssb.detail_target_url,
    ssb.detail_uri,
    ssb.hashtag,
    ssb.id,
    ssb.need_career,
    ssb.need_doc,
    ssb.recruit_title,
    ssb.search_word,
    ssb.target_name,
    ssb.target_url,
    ssb.task,
    ssb.team_env
from
    SCRAPPING_SITE_B ssb''') 
